(complementary:amr)=
# Error estimation & adaptive refinement

In this tutorial, we apply a Zienkiewicz-Zhu type error estimator and run an adaptive loop with these steps:
$$
\text{SOLVE}\rightarrow
\text{ESIMATE}\rightarrow
\text{MARK}\rightarrow
\text{REFINE}\rightarrow
\text{SOLVE} \rightarrow \ldots
$$

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np

### Geometry

The following geometry represents a heated chip embedded in another material that conducts away the heat.

In [ ]:
def MakeGeometryOCC():
    square = Circle( (0,0), 2).Face() # create a rectangle with boundary conditions
    square.edges.name = "outer"
    square.faces.name = "air"

    el1 = MoveTo(-0.4, 0.2).Rectangle( 0.8, 0.1).Face() # create a rectangle with boundary conditions
    el1.edges.name = "el1"
    el1.vertices.name = "el1"

    el2 = MoveTo(-0.4, -0.2).Rectangle( 0.8, 0.1).Face() # create a rectangle with boundary conditions
    el2.edges.name = "el2"
    el2.vertices.name = "el2"

    dielec = MoveTo(-0.9, -0.1).Rectangle( 1.8, 0.3).Face()
    dielec.faces.name = "dielec"

    air = square  # subtract the rectangles from the air rectangle
    shape = Glue([air - dielec, dielec])
    shape =shape - el1 - el2

    ### adding extra specifications on the shape
    #predefined mesh size for the shape


    return OCCGeometry(shape, dim=2)

mesh = Mesh(MakeGeometryOCC().GenerateMesh(maxh=0.5))
mesh.Curve(3)
Draw(mesh);

### Spaces & forms

The problem is to find $u$ in $H_{0,D}^1$ satisfying 

$$
\int_\Omega \varepsilon \nabla u \cdot \nabla v = \int_\Omega f v 
$$

for all $v$ in $H_{0,D}^1$. We expect the solution to have singularities due to the nonconvex re-enrant angles and discontinuities in $\lambda$.

**On the notes adaptivity is explained using $\lambda$ instead of $\varepsilon$.**

In [ ]:
fes = H1(mesh, order=3, dirichlet="el.*")
u, v = fes.TnT()

# one heat conductivity coefficient per sub-domain
eps = CoefficientFunction([1, 2])
a = BilinearForm(eps*grad(u)*grad(v)*dx)

# heat-source in inner subdomain
f = LinearForm(fes)

c = Preconditioner(a, type="multigrid", inverse="sparsecholesky")

electrode = mesh.BoundaryCF({"el1":1, "el2":-1}, default = 0 ) # define the boundary conditions
gfu = GridFunction(fes)
gfu.Set(electrode, definedon=mesh.Boundaries("el.*")) # set the boundary conditions


Note that the linear system is not yet assembled above.

### Solve 

Since we must solve multiple times, we define a function to solve the boundary value problem, where assembly, update, and solve occurs.

In [ ]:
def SolveBVP(gfu):
    fes.Update()
    gfu.Update()
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data +=inv* (f.vec - a.mat * gfu.vec) # solve the system


In [ ]:
SolveBVP(gfu)
Draw(gfu);

### Estimate

We implement a gradient-recovery-type error estimator. For this, we need an H(div) space for flux recovery. We must compute the flux  of the computed solution and interpolate it into this H(div) space.

In [ ]:
space_flux = HDiv(mesh, order=2)
#space_flux = VectorH1(mesh, order=2)

gf_flux = GridFunction(space_flux, "flux")

flux = eps * grad(gfu)
gf_flux.Set(flux)

**Element-wise error estimator:** On each element $T$, set 

$$
\eta_T^2 = \int_T \frac{1}{\varepsilon} 
|\varepsilon \nabla u_h - I_h(\varepsilon \nabla u_h) |^2
$$

where $u_h$ is the computed solution `gfu` and $I_h$ is the interpolation performed by `Set` in NGSolve.


In [ ]:
err = 1/eps*(flux-gf_flux)*(flux-gf_flux)
Draw(err, mesh, 'error_representation')

In [ ]:
eta2 = Integrate(err, mesh, VOL, element_wise=True)

# print only the first 10 values
print(np.array(eta2)[:10], "...")

The above values, one per element, lead us to identify elements which might have large error.


### Mark 

We mark elements with large error estimator for refinement.

In [ ]:
maxerr = max(eta2)
print ("maxerr = ", maxerr)

for el in mesh.Elements():
    mesh.SetRefinementFlag(el, eta2[el.nr] > 0.25*maxerr)


### Refine & solve again 

Refine marked elements:

In [ ]:
mesh.Refine()
SolveBVP(gfu)
Draw(gfu);

### Automate the above steps

In [ ]:
l = []    # l = list of estimated total error

def CalcError():

    # compute the flux:
    space_flux.Update()      
    gf_flux.Update()
    flux = eps * grad(gfu)        
    gf_flux.Set(flux) 
    
    # compute estimator:
    err = 1/eps*(flux-gf_flux)*(flux-gf_flux)
    eta2 = Integrate(err, mesh, VOL, element_wise=True)
    maxerr = max(eta2)
    l.append ((fes.ndof, sqrt(sum(eta2))))
    print("ndof =", fes.ndof, " maxerr =", maxerr)
    
    # mark for refinement (vectorized alternative)
    mesh.ngmesh.Elements2D().NumPy()["refine"] = eta2.NumPy() > 0.25*maxerr
    

In [ ]:
CalcError()
mesh.Refine()
mesh.Curve(4);

### Run the adaptive loop

In [ ]:
level = 0 
while fes.ndof < 100000:  
    SolveBVP(gfu)
    level = level + 1
    if level%5 == 0:
        print('adaptive step #', level)
        Draw(gfu)
    CalcError()
    mesh.Refine()
    mesh.Curve(4)

SolveBVP(gfu)



### Plot history of adaptive convergence

In [ ]:
plt.yscale('log')
plt.xscale('log')
plt.xlabel("ndof")
plt.ylabel("H1 error-estimate")
ndof,err = zip(*l)
plt.plot(ndof,err, "-*")

plt.plot(ndof, [(ndof[0]/n)**(1.5)for n in ndof], "--", label="3/2-th order")

# plot the convergence rate line

plt.legend()
plt.ion()
plt.show()

In [ ]:
N = 10
# fac = 0 if mesh.dim == 2 else 1
# p = [(-1+4*i/N,-2+4*j/N,fac * 2*k/N) for i in range(1,2*N) for j in range(1,2*N) for k in range(1,N)]
p = [(i / N, -1 + (2*j) / N, 0) for i in range(N+1) for j in range(N+1)]

fieldlines = flux._BuildFieldLines(
    mesh, p, num_fieldlines=N**3//5, randomized=False, length=0.3)
clipping = {"clipping": {"y": 0, "z": -1}}


Draw(-eps*grad(gfu), mesh, "X" , min = 0, max = 10);
Draw(-eps*grad(gfu), mesh,  "X", draw_vol=True, draw_surf=True, objects=[fieldlines],
     autoscale=True, min=0, max=10, settings={"Objects": {"Surface": False}}, **clipping);